In [1]:
import time
import tracemalloc
import csv

In [2]:
"""Needleman-Wunch"""
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
  i,j = len(v), len(w)
  new_v = []
  new_w = []
  while True:
      di, dj = pointers[i][j]
      if (di,dj) == LEFT:
          new_v.append('-')
          new_w.append(w[j-1])
      elif (di,dj) == UP:
          new_v.append(v[i-1])
          new_w.append('-')
      elif (di,dj) == TOPLEFT:
          new_v.append(v[i-1])
          new_w.append(w[j-1])
      i, j = i + di, j + dj
      if (i <= 0 and j <= 0):
          break
  return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])

def global_align(v, w, delta):
  """
  Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as
  computed by traceback_global.

  :param: v
  :param: w
  :param: delta
  """
  start_time = time.time()
  tracemalloc.start()
  
  M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
  pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
  score, alignment = None, None

  # YOUR CODE HERE
  M[0][0] = 0;
  for i in range(1, len(v) + 1):
    cur_v = v[i - 1]
    M[i][0] = M[i - 1][0] + delta[cur_v]["-"]
    pointers[i][0] = UP

  for i in range(1, len(w) + 1):
    cur_w = w[i - 1]
    M[0][i] = M[0][i - 1] + delta["-"][cur_w]
    pointers[0][i] = LEFT

  for i in range(1, len(v) + 1):
    for j in range(1, len(w) + 1):
      cur_v = v[i - 1]
      cur_w = w[j - 1]

      left_score = M[i][j - 1] + delta["-"][cur_w]
      top_score = M[i - 1][j] + delta[cur_v]["-"]

      M[i][j] = max(left_score, M[i - 1][j - 1] + delta[cur_v][cur_w], top_score)

      if M[i][j] == left_score:
        pointers[i][j] = LEFT
      elif M[i][j] == top_score:
        pointers[i][j] = UP
      else:
        pointers[i][j] = TOPLEFT

  score = M[len(v)][len(w)]

  alignment = traceback_global(v,w, pointers)

  # Stop memory tracking and calculate time elapsed
  end_time = time.time()
  current, peak = tracemalloc.get_traced_memory()
  tracemalloc.stop()
  elapsed_time = end_time - start_time

  print(f"Time taken: {elapsed_time:.4f} seconds")
  print(f"Current memory usage: {current / 10**6:.6f} MB")
  print(f"Peak memory usage: {peak / 10**6:.6f} MB")

  return alignment, elapsed_time, current, peak

In [3]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

a, b, c, d = hirschberg("T", "ATT", delta)
print(a)

NameError: name 'hirschberg' is not defined

In [23]:
"""
Teating for both algorithms
csv input
"""

import csv

def test_alignment(csv_file, delta, method="global"):
    """
    Tests sequence alignment using either global alignment or Hirschberg's algorithm.

    Parameters:
        csv_file (str): Path to the CSV file containing sequence pairs.
        delta (int): Gap penalty.
        method (str): The alignment method to use ("global" or "hirschberg").

    Returns:
        None
    """
    # Load the sequences from the CSV file
    sequences = []
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            sequences.append((row['sequence1'], row['sequence2']))

    # Function mapping for alignment methods
    alignment_functions = {
        "global": global_align,
        "hirschberg": hirschberg
    }

    # Check if the method is valid
    if method not in alignment_functions:
        raise ValueError(f"Invalid method '{method}'. Choose 'global' or 'hirschberg'.")

    align_function = alignment_functions[method]

    # Test on the first 10 rows
    row_list = []
    for idx, (seq1, seq2) in enumerate(sequences):
        print(f"Test {idx + 1} ({method.capitalize()} Alignment):")
        aligned_seq, elapsed_time, current_mem, peak_mem = align_function(seq1, seq2, delta)
        current_memory = current_mem / 10**6  # Convert bytes to MB
        peak_memory = peak_mem / 10**6  # Convert bytes to MB

        print(f"Aligned Seq: {aligned_seq}\n")

        row_list.append([aligned_seq, elapsed_time, current_memory, peak_memory])

    # Write results to CSV
    output_file = f"alignment_results_{method}.csv"
    with open(output_file, 'w', newline='') as file: 
        writer = csv.writer(file)
        # Write header row
        writer.writerow(['Sequences', 'Elapsed Time (s)', 'Current Memory (MB)', 'Peak Memory (MB)'])
        # Write to CSV
        writer.writerows(row_list)

    print(f"Results saved to {output_file}")


In [256]:
"""
Test Needleman
"""

if __name__ == "__main__":
    # Define the scoring matrix (delta)
    keys = ['A', 'C', 'T', 'G', '-']
    delta = {}

    for i in range(len(keys)):
        delta[keys[i]] = {
            k: (0 if keys[i] == '-' and k == '-' else (1 if keys[i] == k else -1))
            for k in keys
        }
    # Replace with the path to your CSV file
    csv_file = "dataset/rv11_sequence_pairs.csv"
    test_alignment(csv_file, delta)

Test 1 (Global Alignment):
Time taken: 0.0087 seconds
Current memory usage: 0.173993 MB
Peak memory usage: 0.176170 MB
Aligned Seq: AAGAAAG-TTAGAT-ATCAG-ATGTGCTG-TT-AA-AGTGTTTGGTACGGGAAAATGTATAGGCGAGCTAGCCT--GCATAA
AA-AAAGGTTAGATGAGAAGCA-GTAATCCTTGAACA-TGATAGGAAA---AAAA-GAA-ATGA-A-CT-GACTCAGAAAAA

Test 2 (Global Alignment):
Time taken: 0.0089 seconds
Current memory usage: 0.219144 MB
Peak memory usage: 0.221393 MB
Aligned Seq: AA-GAAAGTTAG--ATATCAGATGTGCTG---TTA-AAGTG-TT-T-GGTACGGGAA----AATGTATAGGCGA-GCTAGCCT-GCAT-AA
CGTGAAA-TTCGGGATA-CAGA-GT-CAGGCATTCGAAG-GATTATTGGTAGGGGGGCGCTAA-G-ATAAGCGATGAT-G--TAGTTTTAA

Test 3 (Global Alignment):
Time taken: 0.0078 seconds
Current memory usage: 0.202350 MB
Peak memory usage: 0.204879 MB
Aligned Seq: AAGAAAGTTA-GATA-TCA-G-ATGTG--C-TGTTAA--AG-TGTTTG-GTACGGGA-AAATG--TA--TA---G-GCGAGCTAGCCTGCATAA
--GAAA-TTATG-TGGTAAAGTAAGCGAACCTGTCACGGAGATGAT-GAGAA-GG-ATAAA-GAATAAATAAAAGTG-G-G--AGC--G-AGA-

Test 4 (Global Alignment):
Time taken: 0.0093 seconds
Current

In [261]:
"""
Test Hirschberg
"""

if __name__ == "__main__":
    # Define the scoring matrix (delta)
    keys = ['A', 'C', 'T', 'G', '-']
    delta = {}

    for i in range(len(keys)):
        delta[keys[i]] = {
            k: (0 if keys[i] == '-' and k == '-' else (1 if keys[i] == k else -1))
            for k in keys
        }
    # Replace with the path to your CSV file
    csv_file = "dataset/rv11_sequence_pairs.csv"
    test_alignment(csv_file, delta, "hirschberg")

Test 1 (Hirschberg Alignment):


IndexError: string index out of range

In [ ]:
"""
The alignment score for both alogrithms seems to match!! (it works??)

Chatgpt assisted in writing the code

Major changes:
- small_align got some logic changes
- i added a check_left_tree scenario
- Backward function got some indices changes, but i am not sure if it is correct
- Delta function is changed
"""

import time
import tracemalloc

class TreeNode:  # Customized nodes for the Hirschberg recursion tree
    def __init__(self, indices, i_star, j_split, left_child=None, right_child=None):
        self.indices = indices
        self.i_star = i_star
        self.j_split = j_split
        self.left_child = left_child
        self.right_child = right_child

    def __repr__(self, level=0):  # Recursive tree visualization
        tree = "   " * level + f"{self.indices} ({self.i_star}, {self.j_split})\n"
        if self.left_child:
            tree += self.left_child.__repr__(level + 1)
        if self.right_child:
            tree += self.right_child.__repr__(level + 1)
        return tree


def forward(v, w, delta):
    len_w = len(w); len_v = len(v)
    M = [[None for j in range(len_w)] for i in range(len_v)]
    M[0][0] = 0
    for i in range(1, len_v):
        M[i][0] = M[i-1][0] + delta[v[i]]['-']
    for j in range(1, len_w):
        for i in range(0, len_v):
            if (i == 0):
                M[i][j] = M[i][j-1] + delta[w[j]]['-']
            else: 
                left_score = M[i][j - 1] + delta['-'][w[j]]
                top_score = M[i-1][j] + delta[v[i]]['-']
                M[i][j] = max(left_score, M[i-1][j-1] + delta[v[i]][w[j]], top_score)
            #clear value from colum no longer in use 
            if (j >= 2):
                M[i][j-2] = None
    columns = list(zip(*M))
    return columns[len_w - 1]

def backward(v, w, delta):
    len_w = len(w); len_v = len(v)
    M = [[None for j in range(len_w)] for i in range(len_v)]
    M[-1][-1] = 0
    for i in range(len_v - 2, -1, -1):
        M[i][len_w - 1] = M[i + 1][len_w-1] + delta[v[i]]['-']
    for j in range(len_w - 2, -1, -1):
        for i in range(len_v - 1, -1, -1):
            if (i == len(v) - 1):
                M[i][j] = M[i][j+1] + delta[v[i]]['-']
            else:
                """
                I changed the i and j into i+1 and j+1
                """
                left_score = M[i][j+1] + delta['-'][w[j+1]]
                top_score = M[i+1][j] + delta[v[i+1]]['-']
                M[i][j] = max(left_score, M[i+1][j+1] + delta[v[i+1]][w[j+1]], top_score)
            #clear value from column no longer in use
            if (j + 2 < len(w)):
                M[i][j + 2] = None
    columns = list(zip(*M))
    return columns[0]
    
def hirschberg_recurse(v, w, delta, i, j, i_prime, j_prime):
    v_adjusted = "-" + v
    w_adjusted = "-" + w
    if j_prime - j > 1:
        col_split = j + (j_prime - j) // 2
        prefixes = forward(v_adjusted[i:i_prime + 1], w_adjusted[j:col_split + 1], delta)
        suffixes = backward(v_adjusted[i:i_prime + 1], w_adjusted[col_split:j_prime + 1], delta)
        weights = [p + s for p, s in zip(prefixes, suffixes)]
        i_star = weights.index(max(weights))
        
        left = hirschberg_recurse(v, w, delta, i, j, i_star + i, col_split)
        right = hirschberg_recurse(v, w, delta, i_star + i, col_split, i_prime, j_prime)
        return TreeNode((i, j, i_prime, j_prime), i_star, col_split, left, right)
    else:
        """ 
        I changed the child into NULL pointer
        """
        return TreeNode((i, j, i_prime, j_prime), None, None)

def short_align(v_sub, w_sub):
    """
    Perform optimal alignment for short sequences using dynamic programming.
    This part is changed to figured out how to best align one character with a seq 
    (eg: A, TAT --> -A-, TAT)
    """
    len_v, len_w = len(v_sub), len(w_sub)
    
    # Initialize the scoring matrix for short sequences
    dp = [[0] * (len_w + 1) for _ in range(len_v + 1)]
    
    # Fill the scoring matrix
    for i in range(1, len_v + 1):
        dp[i][0] = dp[i - 1][0] + delta[v_sub[i - 1]]['-']
    for j in range(1, len_w + 1):
        dp[0][j] = dp[0][j - 1] + delta['-'][w_sub[j - 1]]
    for i in range(1, len_v + 1):
        for j in range(1, len_w + 1):
            match = dp[i - 1][j - 1] + delta[v_sub[i - 1]][w_sub[j - 1]]
            delete = dp[i - 1][j] + delta[v_sub[i - 1]]['-']
            insert = dp[i][j - 1] + delta['-'][w_sub[j - 1]]
            dp[i][j] = max(match, delete, insert)

    # Backtrack to construct alignment
    v_align, w_align = [], []
    i, j = len_v, len_w
    while i > 0 or j > 0:
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + delta[v_sub[i - 1]][w_sub[j - 1]]:
            v_align.append(v_sub[i - 1])
            w_align.append(w_sub[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + delta[v_sub[i - 1]]['-']:
            v_align.append(v_sub[i - 1])
            w_align.append("-")
            i -= 1
        else:
            v_align.append("-")
            w_align.append(w_sub[j - 1])
            j -= 1

    # Reverse the alignments and return as strings
    return "".join(reversed(v_align)), "".join(reversed(w_align))

 

def traceback_hirschberg(v, w, node, is_left_tree): 
    if not node:
        return "", ""
    i, j, i_prime, j_prime = node.indices

    # Base cases for empty sequences
    if i_prime - i == 0:
        return "-" * (j_prime - j), w[j:j_prime]
    if j_prime - j == 0:
        return v[i:i_prime], "-" * (i_prime - i)

    # Special cases for short sequences
    if i_prime - i == 1 or j_prime - j == 1:
        return short_align(v[i:i_prime], w[j:j_prime])

    # Recursive traceback for subtrees
    """
    I added a left and right difference, but i have no idea if this matter
    """
    if is_left_tree:
        v_left, w_left = traceback_hirschberg(v, w, node.left_child, True)
        v_right, w_right = traceback_hirschberg(v, w, node.right_child, False)
    else:
        v_right, w_right = traceback_hirschberg(v, w, node.right_child, False)
        v_left, w_left = traceback_hirschberg(v, w, node.left_child, True)


    return v_left + v_right, w_left + w_right


def hirschberg(v, w, delta):
    tracemalloc.start()
    start_time = time.time()

    root = hirschberg_recurse(v, w, delta, 0, 0, len(v), len(w))
    v_align, w_align = traceback_hirschberg(v, w, root, True)

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    elapsed_time = end_time - start_time
    alignment = f"{w_align}\n{v_align}"

    # Performance logging
    print(f"Memory usage:a Current = {current / 10**6:.6f} MB, Peak = {peak / 10**6:.6f} MB")
    print(f"Elapsed time: {elapsed_time:.6f} seconds")

    return alignment, elapsed_time, current, peak


In [ ]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}

for i in range(len(keys)):
    delta[keys[i]] = {
        k: (0 if keys[i] == '-' and k == '-' else (1 if keys[i] == k else -1))
        for k in keys
    }
a, b, c, d = hirschberg("GCAAT", "TTTTTTTTTATTTT", delta)
print(a)


Memory usage:a Current = 0.006198 MB, Peak = 0.006787 MB
Elapsed time: 0.000848 seconds
TTTTTTTTTATTTT
-------GCA--AT
